In [1]:
import numpy as np
from ase.io import read, write
from ase.visualize import view
from ase.data.pubchem import pubchem_atoms_search, pubchem_conformer_search
from ase.build import add_adsorbate
from ase.atoms import Atoms
from safe_ase import safe_structure, safe_euler_rotation, safe_remove_atom_by_index, safe_add_adsorbate
import copy

In [13]:
ed = pubchem_atoms_search(cid=3301)

/home/roberto/anaconda3/envs/dftb/lib/python3.10/site-packages/ase/data/pubchem.py:79: UserWarning: The structure "3301" has more than one conformer in PubChem. By default, the first conformer is returned, please ensure you are using the structure you intend to or use the `ase.data.pubchem.pubchem_conformer_search` function
  warnings.warn('The structure "{}" has more than one '


In [14]:
write('ed.cif', ed)

In [15]:
view(read('ed.cif'))

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [3]:
mono = read('mono4.cif')

In [26]:
view(mono*(2,2,1) + ed)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [35]:
ed.positions[:,2] += np.abs(ed.positions[:,2].min())

In [4]:
def set_to_origin(arr):
    arr[:,0] -= arr[:,0].min()
    arr[:,1] -= arr[:,1].min()
    arr[:,2] -= arr[:,2].min()
    return arr

In [5]:
def move_to(arr, x, y, z):
    arr[:,0] += x
    arr[:,1] += y
    arr[:,2] += z
    return arr

In [37]:
ed.positions = set_to_origin(ed.positions)

In [7]:
ed.positions = move_to(ed.positions, 0.72252, 1.118795, 14)

In [38]:
view(ed)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [72]:
view(mono*(2,1,1))

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [77]:
mono21 = mono*(2,1,1)

In [78]:
mono21.cell

Cell([6.64404, 4.56659, 22.47])

In [9]:
def freeze_atoms(structure: Atoms, freeze_index = []) -> np.array:
    cons = np.zeros((len(structure),3), dtype=int)
    for f in freeze_index:
        cons[f] = 1
    return cons

In [10]:
def pretty_coordinates(structure: Atoms, frozen_atoms = []):
    symbols = np.array(structure.get_chemical_symbols())
    symbols.shape = (len(symbols), 1) # Fix dimensions in order to concatenate
    matched = np.concatenate((symbols, structure.get_positions(), freeze_atoms(structure, frozen_atoms)), axis=1)
    # matched = np.concatenate((symbols, structure.get_positions()), axis=1)
    pretty = ['      '.join(coord) for coord in matched]
    return pretty

In [11]:
def pretty_cell(structure: Atoms, distance: float):
    cell = structure.cell
    cell[2] = cell[2] + [0,0,distance]
    axis = [ c for c in cell]
    return [ '      '.join([str(i) for i in a]) for a in axis] # Doble recursion

In [12]:
from jinja2 import Environment, FileSystemLoader

In [16]:
def build_ed_mono(z_distance, x = 0.72252, y = 1.18795):
    z_max = 12.305 # uppermost postion of p layer
    mono = read('mono4.cif')
    ed = read('ed.cif')
    mono21 = mono*(2,1,1)
    ed.positions = set_to_origin(ed.positions)
    ed.positions = move_to(ed.positions, x, y, z_max + z_distance)
    both_layers = mono21 + ed
    both_layers.cell[2,2] = both_layers.positions[:,2].max() + 5
    return both_layers

In [17]:
def gen_ed_mono_input(template_name: str,
    prefix: str, distance: float,
    frozen_atoms = []
):
    environment = Environment(loader=FileSystemLoader("templates/"))
    template = environment.get_template(template_name)
    both = build_ed_mono(distance)
    s_distance = "{:.2f}".format(distance)
    input_render = {
    "prefijo": prefix + s_distance,
    "dirSalida": prefix + s_distance,
    "nat": len(both),
    "coordenadas": pretty_coordinates(both, frozen_atoms),
    "ejes": pretty_cell(both, distance)
    }
    with open(prefix + s_distance + "-" + template_name,'w',encoding = 'utf-8') as f:
        f.write(template.render(input_render))
        f.close()

In [19]:
view(build_ed_mono(2.0))

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [20]:
gen_ed_mono_input('scf.in', 'ed', 2.3)

In [21]:
np.arange(1.5,3.80,0.05)

array([3.25, 3.3 , 3.35, 3.4 , 3.45, 3.5 , 3.55, 3.6 , 3.65, 3.7 , 3.75])

In [22]:
[gen_ed_mono_input('scf.in', 'ed', i) for i in np.arange(1.5,6.5,0.5)]

[None, None, None, None, None, None, None, None, None, None]

In [44]:
def build_ed_sandwich(z_distance, x = 0.72252, y = 1.18795):
    z_max = 12.305 # uppermost postion of p layer
    delta_ed = 1.770
    delta_intra = 2.137
    lower = read('mono4.cif')
    upper = read('mono4.cif')
    ed = read('ed.cif')
    lower21 = lower*(2,1,1)
    upper21 = upper*(2,1,1)
    ed.positions = set_to_origin(ed.positions)
    ed.positions = move_to(ed.positions, x, y, z_max + z_distance)
    upper21.positions = move_to(upper21.positions, 0, 0, delta_intra + delta_ed + (2 * z_distance))
    three_layers = lower21 + ed + upper21
    three_layers.cell[2,2] = three_layers.positions[:,2].max() + 5
    return three_layers

In [46]:
def gen_ed_sandwich_input(template_name: str,
    prefix: str, distance: float,
    frozen_atoms = []
):
    environment = Environment(loader=FileSystemLoader("templates/"))
    template = environment.get_template(template_name)
    both = build_ed_sandwich(distance)
    s_distance = "{:.2f}".format(distance)
    input_render = {
    "prefijo": prefix + s_distance,
    "dirSalida": prefix + s_distance,
    "nat": len(both),
    "coordenadas": pretty_coordinates(both, frozen_atoms),
    "ejes": pretty_cell(both, distance)
    }
    with open(prefix + s_distance + "-" + template_name,'w',encoding = 'utf-8') as f:
        f.write(template.render(input_render))
        f.close()